# 3.3 线性回归的简洁实现

In [27]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.5.1


## 3.3.1 生成数据集

In [28]:
# 生成数据集，构造线性回归方程
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)

labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
print(features[0], labels[0])

tensor([-1.0964,  2.0029]) tensor(-4.8083)


## 3.3.2 读取数据

In [29]:
# 用PyTorch的data包来读取数据
# 由于data常用作变量名，将导入的data模块用Data代替
# 每一次迭代随机读取包含10个数据样本的小批量
import torch.utils.data as Data
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [30]:
# 读取并打印第一个小批量数据样本
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 1.0617, -0.2833],
        [ 0.9869,  1.0626],
        [-0.2634,  0.9222],
        [ 0.2667,  0.2257],
        [ 0.8574, -0.3547],
        [-0.5448, -1.6379],
        [-0.0068,  0.4947],
        [ 0.6835, -1.5754],
        [ 0.1729, -0.6404],
        [-0.5599, -1.0088]]) tensor([ 7.2985,  2.5514,  0.5615,  3.9606,  7.1203,  8.6798,  2.4975, 10.9069,
         6.7302,  6.4931])


## 3.3.3 定义模型

In [31]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
        # forward定义前向传播
        def forward(self, x):
            y = self.linear(x)
            return y
net = LinearNet(num_inputs)
print(net)  # 使用print可以打印出网络的结构        

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [32]:
# 还可以用nn.Sequential更加方便地搭建网络
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )  
print(net)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


In [33]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# 还可传入其它层，net.add_module...
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [34]:
# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    # 此处还可以传入其他层...
    ]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [35]:
# 通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.5347, 0.7057]], requires_grad=True)
Parameter containing:
tensor([0.6873], requires_grad=True)


## 3.3.4 初始化模型参数

In [36]:
# 在使用net前，需要初始化模型参数
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

## 3.3.5 定义损失函数

In [37]:
# 用nn.Module提供的均方误差损失作为模型的损失函数
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [38]:
# 创建一个用于优化net所有参数的优化器
# 指定小批量随机梯度下降（SGD）为优化算法
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [39]:
# 为不同子网络设置不同的学习率
# optimizer = optim.SGD([
#                  {'params': net.subnet1.parameters()},  # lr=0.03，如果对某个参数不指定学习率，就使用最外层的默认学习率
#                  {'params': net.subnet2.parameters(), 'lr': 0.01}
#                  ], lr=0.03)

In [40]:
# 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1  # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [41]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()  # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()  # 调用optim实例的step函数来迭代模型参数
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000169
epoch 2, loss: 0.000099
epoch 3, loss: 0.000108


In [42]:
# 比较学到的模型参数和真实的模型参数
dense = net[0]  # net[0]表示输入层
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9994, -3.3999]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1998], requires_grad=True)


## 小结

### 使用PyTorch可以更简洁地实现模型。torch.utils.data模块提供了有关数据处理的工具，torch.nn模块定义了大量神经网络的层，torch.nn.init模块定义了各种初始化方法，torch.optim模块提供了很多常用的优化算法。